In [1]:
import brickschema
import pandas as pd
from functions.bacnet_point import BACnet_Point
import functions.readWriteProperty as BACpypesAPP

In [2]:
g = brickschema.Graph()
g.load_file('chamber_shacl_expanded.ttl')

<Graph identifier=Nd33a71e9e8e340ebafd2d6a723b9847a (<class 'brickschema.graph.Graph'>)>

In [3]:
# Query test 1
query1 = g.query(
    """SELECT ?ahu ?damper ?damper_pos WHERE {
    ?ahu a brick:Air_Handling_Unit .
    ?ahu brick:hasPart ?damper .
    ?damper a brick:Damper .
    ?damper brick:feeds ?cc .
    ?cc a brick:Chilled_Water_Coil .
    ?damper_pos a brick:Damper_Position_Sensor .
    ?damper brick:hasPoint ?damper_pos
}"""
)
print("\nQuery test 1 results:")
for row in query1:
    print(row)


Query test 1 results:
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#AHU_C'), rdflib.term.URIRef('http://berkeley.edu/climatechamber#damper_SC_supply'), rdflib.term.URIRef('http://berkeley.edu/climatechamber#spot_clg_dmpr_pos_ao_6'))
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#AHU_C'), rdflib.term.URIRef('http://berkeley.edu/climatechamber#damper_supply'), rdflib.term.URIRef('http://berkeley.edu/climatechamber#supply_damper_pos_10'))


In [4]:
# Query test 2
query2 = g.query(
    """SELECT * WHERE{ 
    ?sensor rdf:type/rdfs:subClassOf* brick:Setpoint .
    ?sensor brick:isPointOf ?equipment
}"""
)
print("\nQuery test 2 results:")
for row in query2:
    print(row)


Query test 2 results:
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#AHU_C'), rdflib.term.URIRef('http://berkeley.edu/climatechamber#dsp_fixed_setpoint_10'))
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#AHU_C'), rdflib.term.URIRef('http://berkeley.edu/climatechamber#press_stpt_6'))
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#AHU_C'), rdflib.term.URIRef('http://berkeley.edu/climatechamber#dsp_fixed_setpoint_10'))
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#AHU_C'), rdflib.term.URIRef('http://berkeley.edu/climatechamber#press_stpt_6'))
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#AHU_C'), rdflib.term.URIRef('http://berkeley.edu/climatechamber#dat_sp_clg_10'))
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#AHU_C'), rdflib.term.URIRef('http://berkeley.edu/climatechamber#dat_sp_clg_10'))
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#WH_1'), rdflib.term.URIRef('http://berkeley.edu/climatechamber#HWST_stp'))
(rdflib.te

In [5]:
# Query test 3
query3 = g.query(
    """SELECT ?position WHERE{
    ?ahu a brick:Air_Handling_Unit .
    ?position a brick:Damper_Position_Sensor .
    ?damper a brick:Damper .
    ?zone a brick:HVAC_Zone .
    ?ahu brick:hasPart ?damper .
    ?damper brick:hasPoint ?position .
    ?damper brick:feeds ?zone
    }
    """
)
print("\nQuery test 3 results:")
for row in query3:
    print(row)


Query test 3 results:
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#spot_clg_dmpr_pos_ao_6'),)


In [6]:
# Query test 4
query4 = g.query(
    """SELECT ?sat WHERE{
        ?ahu a brick:Air_Handling_Unit .
        ?sat a brick:Supply_Air_Temperature_Sensor .
        ?ahu brick:hasPoint ?sat
    }
    """
)
print("\nQuery test 4 results:")
for row in query4:
    print(row)


Query test 4 results:
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#sa_temp_1'),)
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#coil_da_temp_10'),)
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#eff_sa_temp_10'),)
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#spot_clg_sa_temp_6'),)
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#overhead_sa_temp_10'),)
(rdflib.term.URIRef('http://berkeley.edu/climatechamber#underfloor_sa_temp_10'),)


In [7]:
bacnet_ini_file = './readfiles/BACnet_init_controller.ini'
access_bacnet = BACpypesAPP.Init(bacnet_ini_file)

> /Users/aoyuzou/Documents/Mac/ClassMaterial/ARCH299/Brick/chamber/functions/readWriteProperty.py(336)Init()
    334     except Exception as error:
    335         import pdb; pdb.set_trace()
--> 336         if _debug: print("exception: %r", error)
    337     else:
    338         bacnet_establish = True



In [ ]:
brick_point = 'brick:Chilled_Water_Temperature_Setpoint'

query = g.query(
    f""" SELECT * WHERE {{
        VALUES ?t_type {{ {brick_point} }}
             ?equipment brick:hasPoint              ?point .
             ?point     rdf:type/rdfs:subClassOf?   ?t_type .
             ?point     brick:hasUnit               ?point_unit .

             ?point     ref:hasExternalReference    ?ref.
             ?ref       bacnet:object-name          ?obj_name .
             ?ref       bacnet:object-identifier    ?obj_identifier .
             ?ref       bacnet:objectOf             ?obj_device .
             ?obj_device bacnet:hasPort             ?ref_port .
             ?ref_port  ref:storedAt                ?bacnet_address .
        }}"""
)

df_result = pd.DataFrame(query, columns=[str(s) for s in query.vars])

if not df_result.empty:
    setpoint_dict = df_result.to_dict('records')[0]
else:
    setpoint_dict = {}

In [26]:
setpoint_dict

{'obj_device': rdflib.term.URIRef('http://berkeley.edu/climatechamber#dev-2400001.0'),
 't_type': rdflib.term.URIRef('https://brickschema.org/schema/Brick#Chilled_Water_Temperature_Setpoint'),
 'bacnet_address': rdflib.term.Literal('192.168.168.1'),
 'obj_identifier': rdflib.term.Literal('analogValue,2'),
 'obj_name': rdflib.term.Literal('fixed_chws_temp_stpt_1'),
 'point_unit': rdflib.term.URIRef('http://qudt.org/vocab/unit/DEG_F'),
 'ref': rdflib.term.BNode('n9d090620468d4008938bda36c1d0b3c4b22213'),
 'point': rdflib.term.URIRef('http://berkeley.edu/climatechamber#fixed_chws_temp_stpt_1'),
 'equipment': rdflib.term.URIRef('http://berkeley.edu/climatechamber#PCHW_L'),
 'ref_port': rdflib.term.BNode('n9d090620468d4008938bda36c1d0b3c4b24563')}

In [27]:
setpoint = BACnet_Point(**setpoint_dict) if bool(setpoint_dict) else setpoint_dict

BACnet_Point(setpoint_dict['name'], setpoint_dict['unit'], ...)

In [ ]:
BACpypesAPP.write_prop(setpoint_dict['bacnet_address'], setpoint_dict['obj_identifier'][0], setpoint_dict[''])

In [48]:
setpoint.write_point_value(BACpypesAPP, 47, 13)

In [30]:
brick_point = 'brick:Leaving_Chilled_Water_Temperature_Sensor'

query = g.query(
    f""" SELECT * WHERE {{
        VALUES ?t_type {{ {brick_point} }}
             ?equipment brick:hasPoint              ?point .
             ?point     rdf:type/rdfs:subClassOf?   ?t_type .
             ?point     brick:hasUnit               ?point_unit .

             ?point     ref:hasExternalReference    ?ref.
             ?ref       bacnet:object-name          ?obj_name .
             ?ref       bacnet:object-identifier    ?obj_identifier .
             ?ref       bacnet:objectOf             ?obj_device .
             ?obj_device bacnet:hasPort             ?ref_port .
             ?ref_port  ref:storedAt                ?bacnet_address .
        }}"""
)

df_result = pd.DataFrame(query, columns=[str(s) for s in query.vars])

if not df_result.empty:
    temp_dict = df_result.to_dict('records')
else:
    temp_dict = {}

In [31]:
temp_dict

[{'obj_device': rdflib.term.URIRef('http://berkeley.edu/climatechamber#dev-2400002.0'),
  't_type': rdflib.term.URIRef('https://brickschema.org/schema/Brick#Leaving_Chilled_Water_Temperature_Sensor'),
  'bacnet_address': rdflib.term.Literal('192.168.168.2'),
  'obj_identifier': rdflib.term.Literal('analogValue,223'),
  'obj_name': rdflib.term.Literal('supply_temp_b_11'),
  'point_unit': rdflib.term.URIRef('http://qudt.org/vocab/unit/DEG_F'),
  'ref': rdflib.term.BNode('n9d090620468d4008938bda36c1d0b3c4b20002'),
  'point': rdflib.term.URIRef('http://berkeley.edu/climatechamber#supply_temp_b_11'),
  'equipment': rdflib.term.URIRef('http://berkeley.edu/climatechamber#Meter_C'),
  'ref_port': rdflib.term.BNode('n9d090620468d4008938bda36c1d0b3c4b24564')},
 {'obj_device': rdflib.term.URIRef('http://berkeley.edu/climatechamber#dev-2400001.0'),
  't_type': rdflib.term.URIRef('https://brickschema.org/schema/Brick#Leaving_Chilled_Water_Temperature_Sensor'),
  'bacnet_address': rdflib.term.Litera

In [ ]:
supply_temp = BACnet_Point(**temp_dict) if bool(temp_dict) else temp_dict

In [ ]:
supply_temp.get_point_value(BACpypesAPP)